In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import pickle

from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator
from diffusers import (
    PNDMScheduler,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    KDPM2DiscreteScheduler,
    DEISMultistepScheduler,
)

from pts.model.time_grad import TimeGradEstimator
from pts.dataset.repository.datasets import dataset_recipes

from utils import omit_points, interpolate_np_array

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/artificial/_base.py:84: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.freq: BaseOffset = to_offset(freq)


In [3]:
dataset = get_dataset("exchange_rate_nips", regenerate=False)

train_grouper = MultivariateGrouper(
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality)
)

test_grouper = MultivariateGrouper(
    num_test_dates=int(len(dataset.test) / len(dataset.train)),
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
)

dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/common.py:262: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.Period(val, freq)
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:113: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  timestamp + len(data[FieldName.TARGET]) - 1,
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:242: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  index=pd.period_range(
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/multivariate_grouper.py:242: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' inste

In [4]:
evaluator = MultivariateEvaluator(
    quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={"sum": np.sum}
)

In [5]:
def plot(
    target,
    forecast,
    prediction_length,
    prediction_intervals=(50.0, 90.0),
    color="g",
    fname=None,
):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
        50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
    ]

    percentiles_sorted = sorted(set(ps))

    def alpha_for_percentile(p):
        return (p / 100.0) ** 0.3

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:, dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches="tight", pad_inches=0.05)


In [6]:
scheduler = DEISMultistepScheduler(
    num_train_timesteps=150,
    beta_end=0.1,
)

In [7]:
estimator = TimeGradEstimator(
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    hidden_size=64,
    num_layers=2,
    dropout_rate=0.1,
    lags_seq=[1],
    scheduler=scheduler,
    num_inference_steps=149,
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="mean",
    trainer_kwargs=dict(max_epochs=200, accelerator="gpu", devices="1")
)

In [8]:
predictor = estimator.train(dataset_train, cache_data=True, shuffle_buffer_length=1024)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-05-23 02:06:15.224696: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-23 02:06:17.574167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 02:06:23.538873: W tens

Epoch 0: |          | 50/? [00:07<00:00,  6.29it/s, v_num=16, train_loss=0.333]

Epoch 0, global step 50: 'train_loss' reached 0.33257 (best 0.33257), saving model to '/home/jupyter/work/resources/lightning_logs/version_16/checkpoints/epoch=0-step=50.ckpt' as top 1


Epoch 1: |          | 50/? [00:04<00:00, 11.17it/s, v_num=16, train_loss=0.0538]

Epoch 1, global step 100: 'train_loss' reached 0.05377 (best 0.05377), saving model to '/home/jupyter/work/resources/lightning_logs/version_16/checkpoints/epoch=1-step=100.ckpt' as top 1


...

Epoch 198: |          | 50/? [00:04<00:00, 10.85it/s, v_num=16, train_loss=0.00228]

Epoch 198, global step 9950: 'train_loss' was not in top 1


Epoch 199: |          | 50/? [00:04<00:00, 11.54it/s, v_num=16, train_loss=0.00243]

Epoch 199, global step 10000: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: |          | 50/? [00:04<00:00, 11.53it/s, v_num=16, train_loss=0.00243]


In [9]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset_test, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/split.py:181: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  sliced_entry[FieldName.START] += offset
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/transform/feature.py:364: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  index = pd.period_range(start, periods=length, freq=start.freq)
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/transform/feature.py:364: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  index = pd.period_range(start, periods=length, freq=start.freq)
/home/jupyter/.local/lib/python3.10/site-packages/gluonts/transform/split.py:150: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a

In [10]:
print('Timegrad on regular dataset metrics:')
print("CRPS: {}".format(agg_metric["mean_wQuantileLoss"]))
print("ND: {}".format(agg_metric["ND"]))
print("NRMSE: {}".format(agg_metric["NRMSE"]))
print("MSE: {}".format(agg_metric["MSE"]))

print('')

print("CRPS-Sum: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))
print("ND-Sum: {}".format(agg_metric["m_sum_ND"]))
print("NRMSE-Sum: {}".format(agg_metric["m_sum_NRMSE"]))
print("MSE-Sum: {}".format(agg_metric["m_sum_MSE"]))

Timegrad on regular dataset metrics:
CRPS: 0.009804771380244565
ND: 0.012762573180037875
NRMSE: 0.01870405598936431
MSE: 0.00023141310926140098

CRPS-Sum: 0.006831932999672038
ND-Sum: 0.009275056351774532
NRMSE-Sum: 0.010998395960661751
MSE-Sum: 0.005121003141005834


In [11]:
with open('/home/jupyter/datasphere/project/models/exchange.pkl', 'wb') as f:
    pickle.dump(predictor, f)

In [12]:
"""
# If you want to load already prepared model

transformation = estimator.create_transformation()
lightning_module = estimator.create_lightning_module()
predictor = estimator.create_predictor(transformation, lightning_module)
with open('/home/jupyter/datasphere/project/models/exchange.pkl', 'rb') as f:
    predictor = pickle.load(f)
"""

"\n# If you want to load already prepared model\n\ntransformation = estimator.create_transformation()\nlightning_module = estimator.create_lightning_module()\npredictor = estimator.create_predictor(transformation, lightning_module)\nwith open('/home/jupyter/datasphere/project/models/exchange.pkl', 'rb') as f:\n    predictor = pickle.load(f)\n"

In [13]:
def interpolate_points(dataset, num_points: int = 5):
    new_dataset = []
    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(interpolate_np_array, [(ts['target'], num_points) for ts in dataset])
        for i, ts in enumerate(dataset):
            mask = np.isnan(ts['target'])
            new_dataset.append({
                'target': ts['target'] * (1. - mask) + results[i] * mask,
                'start': ts['start'],
                'feat_static_cat': ts['feat_static_cat']
            })
    return new_dataset

In [ ]:
crps_sum_results = pd.DataFrame(columns=range(10))

with tqdm(total=9*10) as pbar:
    for p in np.linspace(0.1, 1, 9, endpoint=False): # frac of removed points
        dataset_test_p = omit_points(dataset_test, p)
        results_p = []

        # No interpolation
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset_test_p, 
            predictor=predictor, 
            num_samples=100,
        )
        forecasts = list(forecast_it)
        # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
        agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
        results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
        pbar.update(1)

        # Interpolation with num_points
        for num_points in range(1, 10):
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=interpolate_points(dataset_test_p, num_points), 
                predictor=predictor, 
                num_samples=100,
            )
            forecasts = list(forecast_it)
            # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
            agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
            results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
            pbar.update(1)

        crps_sum_results.loc[p] = results_p

In [15]:
crps_sum_results

,0,1,2,3,4,5,6,7,8,9
0.1,0.054266,0.054410,0.053888,0.054353,0.054360,0.054267,0.054520,0.054488,0.054429,0.054457
0.2,0.043867,0.043512,0.043665,0.043656,0.043364,0.043717,0.043635,0.043693,0.043603,0.043542
0.3,0.045334,0.044775,0.045092,0.045068,0.045057,0.045079,0.045016,0.044744,0.045183,0.044945
0.4,0.037477,0.037537,0.037361,0.037349,0.037605,0.037802,0.037563,0.037460,0.037297,0.037443
0.5,0.032438,0.032426,0.032415,0.032507,0.032439,0.032579,0.032639,0.032512,0.032375,0.032383
0.6,0.034405,0.034481,0.034650,0.034610,0.034478,0.034623,0.034254,0.034466,0.034350,0.034428
0.7,0.029516,0.029561,0.029201,0.029174,0.029397,0.029534,0.029399,0.029451,0.029494,0.029560
0.8,0.028845,0.029066,0.028885,0.028710,0.028768,0.028853,0.028940,0.029061,0.028885,0.028854
0.9,0.103626,0.103873,0.103842,0.104025,0.104110,0.103664,0.103781,0.104029,0.103925,0.103764
